Code to test power_flow_processing library

In [1]:
import power_flow_processing as pfp
import pandas as pd
from power_grid_model.utils import json_deserialize_from_file
import numpy as np
import scipy as sp

In [2]:
# with open("input_network_data.json") as fp:
#     data = fp.read()
# pprint.pprint(json.loads(data))

dataset = json_deserialize_from_file("input_network_data.json")

print(type(dataset))
print(pd.DataFrame(dataset.keys()))

print(pd.DataFrame(dataset["line"]))
print(pd.DataFrame(dataset["node"]))
print(pd.DataFrame(dataset["source"]))
print(pd.DataFrame(dataset["sym_load"]))

<class 'dict'>
          0
0      node
1      line
2    source
3  sym_load
   id  from_node  to_node  from_status  to_status    r1   x1       c1  tan1  \
0   5          1        2            1          1  0.25  0.2  0.00001   0.0   
1   6          2        3            1          1  0.25  0.2  0.00001   0.0   
2   7          3        4            1          1  0.25  0.2  0.00001   0.0   

   r0  x0  c0  tan0     i_n  
0 NaN NaN NaN   NaN  1000.0  
1 NaN NaN NaN   NaN  1000.0  
2 NaN NaN NaN   NaN  1000.0  
   id  u_rated
0   1  10500.0
1   2  10500.0
2   3  10500.0
3   4  10500.0
   id  node  status  u_ref  u_ref_angle           sk  rx_ratio  z01_ratio
0  11     1       1    1.0          NaN  200000000.0       NaN        NaN
   id  node  status  type  p_specified  q_specified
0   8     2       1     0          0.0          0.0
1   9     3       1     0          0.0          0.0
2  10     4       1     0          0.0          0.0


In [3]:
active_power_data = pd.read_parquet("active_power_profile.parquet")
print(type(active_power_data))

print(active_power_data)

<class 'pandas.core.frame.DataFrame'>
Load_ID                         8              9              10
Timestamp                                                       
2024-01-01 00:00:00   97627.007855  430378.732745  205526.752143
2024-01-01 01:00:00   89766.365994 -152690.401322  291788.226133
2024-01-01 02:00:00 -124825.577475  783546.001564  927325.521002
2024-01-01 03:00:00 -233116.962348  583450.076165   57789.839506
2024-01-01 04:00:00  136089.122188  851193.276585 -857927.883604
2024-01-01 05:00:00 -825741.400597 -959563.205119  665239.691096
2024-01-01 06:00:00  556313.501900  740024.296494  957236.684466
2024-01-01 07:00:00  598317.128433  -77041.275494  561058.352573
2024-01-01 08:00:00 -763451.148262  279842.042655 -713293.425182
2024-01-01 09:00:00  889337.834099   43696.643500 -170676.120019


In [4]:
reactive_power_data = pd.read_parquet("reactive_power_profile.parquet")

print(reactive_power_data)

Load_ID                         8              9              10
Timestamp                                                       
2024-01-01 00:00:00 -470888.775791  548467.378868  -87699.335567
2024-01-01 01:00:00  136867.897737 -962420.399127  235270.994152
2024-01-01 02:00:00  224191.445445  233867.993750  887496.157029
2024-01-01 03:00:00  363640.598207 -280984.198852 -125936.092401
2024-01-01 04:00:00  395262.391855 -879549.056741  333533.430891
2024-01-01 05:00:00  341275.739236 -579234.877852 -742147.404690
2024-01-01 06:00:00 -369143.298152 -272578.458115  140393.540836
2024-01-01 07:00:00 -122796.973075  976747.676118 -795910.378504
2024-01-01 08:00:00 -582246.487810 -677380.964230  306216.650931
2024-01-01 09:00:00 -493416.794920  -67378.454287 -511148.815997


In [5]:
grid1 = pfp.PowerFlow(grid_data=dataset)

output_data = grid1.batch_powerflow(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)

print(grid1.aggregate_voltage_table(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data))

                     Max_Voltage  Max_Voltage_Node  Min_Voltage  \
Timestamp                                                         
2024-01-01 00:00:00     1.004847                 1     1.003450   
2024-01-01 01:00:00     1.012053                 3     1.007998   
2024-01-01 02:00:00     0.997474                 1     0.984365   
2024-01-01 03:00:00     1.006557                 4     1.005190   
2024-01-01 04:00:00     1.011007                 4     1.005877   
2024-01-01 05:00:00     1.020486                 4     1.010570   
2024-01-01 06:00:00     1.006342                 1     0.998868   
2024-01-01 07:00:00     1.004306                 1     1.002822   
2024-01-01 08:00:00     1.020402                 4     1.010501   
2024-01-01 09:00:00     1.015742                 4     1.010084   

                     Min_Voltage_Node  
Timestamp                              
2024-01-01 00:00:00                 3  
2024-01-01 01:00:00                 1  
2024-01-01 02:00:00                

In [6]:
voltage_table = pd.DataFrame()

voltage_table["Timestamp"] = active_power_data.index.tolist()
voltage_table["max_id"] = output_data["node"][
    0, pd.DataFrame(output_data["node"]["u_pu"][:, :]).idxmax(axis=1).tolist()
]["id"]
voltage_table["u_pu_max"] = pd.DataFrame(output_data["node"]["u_pu"][:, :]).max(axis=1).tolist()
voltage_table["min_id"] = output_data["node"][
    0, pd.DataFrame(output_data["node"]["u_pu"][:, :]).idxmin(axis=1).tolist()
]["id"]
voltage_table["u_pu_min"] = pd.DataFrame(output_data["node"]["u_pu"][:, :]).min(axis=1).tolist()


print(pd.DataFrame(voltage_table))

            Timestamp  max_id  u_pu_max  min_id  u_pu_min
0 2024-01-01 00:00:00       1  1.004847       3  1.003450
1 2024-01-01 01:00:00       3  1.012053       1  1.007998
2 2024-01-01 02:00:00       1  0.997474       4  0.984365
3 2024-01-01 03:00:00       4  1.006557       1  1.005190
4 2024-01-01 04:00:00       4  1.011007       1  1.005877
5 2024-01-01 05:00:00       4  1.020486       1  1.010570
6 2024-01-01 06:00:00       1  1.006342       4  0.998868
7 2024-01-01 07:00:00       1  1.004306       3  1.002822
8 2024-01-01 08:00:00       4  1.020402       1  1.010501
9 2024-01-01 09:00:00       4  1.015742       1  1.010084


In [7]:
pd.DataFrame(output_data["node"]["u_pu"][:, :])

,0,1,2,3
0,1.004847,1.004776,1.003450,1.003459
1,1.007998,1.010113,1.012053,1.011294
2,0.997474,0.992899,0.987828,0.984365
3,1.005190,1.005913,1.006144,1.006557
4,1.005877,1.007426,1.009366,1.011007
5,1.010570,1.016381,1.020329,1.020486
6,1.006342,1.003665,1.000984,0.998868
7,1.004306,1.003310,1.002822,1.003303
8,1.010501,1.016461,1.019042,1.020402
9,1.010084,1.011861,1.014130,1.015742


In [8]:
output_data["node"][:, pd.DataFrame(output_data["node"]["u_pu"][:, :]).idxmax(axis=1).tolist()]["id"]

array([[1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4],
       [1, 3, 1, 4, 4, 4, 1, 1, 4, 4]])

In [9]:
pd.DataFrame(output_data["node"]["u_pu"][:, :]).idxmax(axis=1)

0    0
1    2
2    0
3    3
4    3
5    3
6    0
7    0
8    3
9    3
dtype: int64

In [10]:
grid1 = pfp.PowerFlow(grid_data=dataset)

output_data = grid1.batch_powerflow(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)

print(grid1.aggregate_loading_table(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data))

         Total_Loss  Max_Loading Max_Loading_Timestamp  Min_Loading  \
Line_ID                                                               
5         63.294763     0.149830   2024-01-01 06:00:00     0.063798   
6         36.775143     0.111039   2024-01-01 05:00:00     0.037184   
7         14.872359     0.071700   2024-01-01 02:00:00     0.020380   

        Min_Loading_Timestamp  
Line_ID                        
5         2024-01-01 03:00:00  
6         2024-01-01 00:00:00  
7         2024-01-01 01:00:00  


In [11]:
expected_output1 = pd.read_parquet("output_table_row_per_line.parquet")
expected_output2 = pd.read_parquet("output_table_row_per_timestamp.parquet")

print(expected_output1)
print(expected_output2)

         Total_Loss  Max_Loading Max_Loading_Timestamp  Min_Loading  \
Line_ID                                                               
5         63.294763     0.149830   2024-01-01 06:00:00     0.063798   
6         36.775143     0.111039   2024-01-01 05:00:00     0.037184   
7         14.872359     0.071700   2024-01-01 02:00:00     0.020380   

        Min_Loading_Timestamp  
Line_ID                        
5         2024-01-01 03:00:00  
6         2024-01-01 00:00:00  
7         2024-01-01 01:00:00  
                     Max_Voltage  Max_Voltage_Node  Min_Voltage  \
Timestamp                                                         
2024-01-01 00:00:00     1.004847                 1     1.003450   
2024-01-01 01:00:00     1.012053                 3     1.007998   
2024-01-01 02:00:00     0.997474                 1     0.984365   
2024-01-01 03:00:00     1.006557                 4     1.005190   
2024-01-01 04:00:00     1.011007                 4     1.005877   
2024-01-01 05:0

In [12]:
grid1 = pfp.PowerFlow(grid_data=dataset)

output_data = grid1.batch_powerflow(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)

line_data = output_data["line"]
line_ids = line_data["id"][0, :]

# Extract power data
p_from = pd.DataFrame(line_data["p_from"][:, :], columns=line_ids)
p_to = pd.DataFrame(line_data["p_to"][:, :], columns=line_ids)

# Calculate power loss
p_loss = (p_from + p_to) * 1e-3

# Calculate energy loss
e_loss = sp.integrate.trapezoid(p_loss, dx=1.0, axis=0)

print(e_loss)

[63.29476348 36.77514305 14.87235855]


In [13]:
grid1 = pfp.PowerFlow(grid_data=dataset)


print(grid1.aggregate_loading_table(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data))

         Total_Loss  Max_Loading Max_Loading_Timestamp  Min_Loading  \
Line_ID                                                               
5         63.294763     0.149830   2024-01-01 06:00:00     0.063798   
6         36.775143     0.111039   2024-01-01 05:00:00     0.037184   
7         14.872359     0.071700   2024-01-01 02:00:00     0.020380   

        Min_Loading_Timestamp  
Line_ID                        
5         2024-01-01 03:00:00  
6         2024-01-01 00:00:00  
7         2024-01-01 01:00:00  


Error testing

In [14]:
from power_grid_model import PowerGridModel, initialize_array
import power_flow_processing as pfp

# A power grid containing several errors

# node
node_error = initialize_array("input", "node", 3)
node_error["id"] = [1, 2, 3]
node_error["u_rated"] = [10.5e3]

# line
line_error = initialize_array("input", "line", 3)
line_error["id"] = [4, 5, 6]
line_error["from_node"] = [1, 2, 3]
line_error["to_node"] = [2, 3, 4]
line_error["from_status"] = [True]
line_error["to_status"] = [True]
line_error["r1"] = [0.25]
line_error["x1"] = [0.2]
line_error["c1"] = [10e-6]
line_error["tan1"] = [0.0]

# Power Sensor
sensor_error = initialize_array("input", "sym_power_sensor", 2)
sensor_error["id"] = [6, 7]
sensor_error["measured_object"] = [3, 4]
sensor_error["measured_terminal_type"] = [0, 2]
sensor_error["p_measured"] = [0]
sensor_error["q_measured"] = [0]
sensor_error["power_sigma"] = [0]

error_data = {"node": node_error, "line": line_error, "sym_power_sensor": sensor_error}

pfp.PowerFlow(grid_data=error_data)

ValidationException: There are 2 validation errors in input_data:
   1. Fields line.id and sym_power_sensor.id are not unique for 2 lines/sym_power_sensors.
   2. Field 'to_node' does not contain a valid node id for 1 line.